In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, find_peaks_cwt
from SecondaryValue import SecondaryValue
from scipy.optimize import curve_fit

In [2]:
komm_1 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/kommerzieller_HeNe/4.1spektr.txt',
            skiprows=4)
komm_p1 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/kommerzieller_HeNe/pol1.txt',
            skiprows=1) * 1e4
komm_p2 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/kommerzieller_HeNe/pol2.txt',
            skiprows=4)


In [3]:
komm_1_narrowed = komm_1[20:250]

In [4]:
peaks, peak_info = find_peaks(komm_1_narrowed[:,1], height=80, width=1, rel_height=.5)
all_peaks, _ = find_peaks(komm_1[:,1], height=60, distance=10)

In [5]:
%matplotlib qt5
plt.plot(*komm_1_narrowed.T, label='Spektrum')
plt.plot(komm_1_narrowed[:,0][peaks[1::2]], komm_1_narrowed[:,1][peaks[1::2]], "x",
         color='red', label='Peaks')
plt.xlabel('Zeit [u]')
plt.ylabel('Amplitude [mV]')
plt.legend()
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [6]:
dt = .5
fsr = 1.99862e9, 0.067205e9

In [7]:
peak_t = komm_1_narrowed[:,0][peaks]

In [8]:
unit = SecondaryValue('f/(x2-x1)')(f=fsr, x2=(peak_t[3], 1), x1=(peak_t[1], 1))


In [9]:
unit

(17229482.75862069, 616257.0518525749)

In [57]:
def plot_peaks(ydata, peaks):
    xdata = np.arange(0, len(ydata))

    plt.clf()
    plt.plot(xdata, ydata, label='Spektrum')
    plt.plot(xdata[peaks], ydata[peaks], "x",
             color='red', label='Peaks')
    plt.xlabel('Zeit [u]')
    plt.ylabel('Amplitude [mV]')
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.show()
    
def calibrate_unit(ydata, thresh):
    peaks, peak_info = find_peaks(ydata, height=thresh, width=1, rel_height=.5)
    plot_peaks(ydata, peaks)
    return SecondaryValue('f/(x2-x1)')(f=fsr, x2=(peaks[1], 1), x1=(peaks[0], 1))




In [11]:
calibrate_unit(komm_1_narrowed[:,1], 120)

(17229482.75862069, 616257.0518525749)

In [12]:
widths = peak_info['widths']
width = widths.mean()
d_width = widths.std()/2

In [13]:
width, d_width

(4.7190589321125325, 0.30525965381031983)

In [14]:
w = SecondaryValue('f*u')(f=(width, d_width), u=unit)

In [15]:
w

(81306944.50774784, 6009936.594974544)

In [16]:
fin = SecondaryValue('fsr/fw')(fsr=fsr, fw=w)

In [17]:
fin

(24.581172150793947, 1.9961301696025398)

In [18]:
plt.clf()
p1_fixed = komm_p1-komm_p1.min()
plt.plot(np.arange(0, len(komm_p1)), p1_fixed/p1_fixed.max())
plt.plot(np.arange(0, len(komm_p2)), komm_p2/komm_p2.max())
plt.show()

In [19]:
%matplotlib qt5
plt.plot(*komm_1.T, label='Spektrum')
plt.plot(komm_1[:,0][all_peaks], komm_1[:,1][all_peaks], "x",
         color='red', label='Peaks')
plt.xlabel('Zeit [u]')
plt.ylabel('Amplitude [mV]')
plt.legend()
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [33]:
dists = all_peaks[1::2] - all_peaks[0::2]
dist = dists.mean(), dists.std()/np.sqrt(len(dists))
real_dist = SecondaryValue('u*d')(u=unit, d=dist)

In [34]:
dist

(37.6, 2.2199099080818567)

In [35]:
real_dist

(647828551.724138, 44719227.898102246)

In [36]:
L = SecondaryValue('c/(2*d)')(c=299792458, d=real_dist)

In [37]:
L

(0.23138256040284816, 0.015972203483102267)

In [53]:
real_dist

(21594231.03448276, 1301824.6072136145)

In [38]:
dist

(37.6, 2.2199099080818567)

In [39]:
all_peaks[1::2]

array([ 88, 204, 313, 443, 574])

In [59]:
def find_distances(ydata, thresh, num_peaks, new_unit):
    all_peaks, _ = find_peaks(ydata, height=thresh, distance=10)

    num_batches = len(all_peaks) / num_peaks
    peak_batches = np.split(all_peaks, num_batches)

    
    plot_peaks(ydata, all_peaks)
    dists = np.concatenate([d[1:] - d[:-1] for d in peak_batches])
    dist = dists.mean(), dists.std()/np.sqrt(len(dists))
    return SecondaryValue('u*d')(u=new_unit, d=dist)
mode_dist = SecondaryValue('c/(2*L)', defaults=dict(c=29979245800))


In [60]:
off_1 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/offener_HeNe/Waveform.txt',
            skiprows=5)
unit_1 = calibrate_unit(off_1, 79)


In [51]:
find_distances(off_1, 50, 3, unit_1)


(200825964.6302251, 14256332.17614453)

In [52]:
mode_dist(L=(80,.5))

(187370286.25, 1171064.2890625)

In [53]:
off_2 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/offener_HeNe/res_laenge_60cm.txt',
            skiprows=5)
unit_2 = calibrate_unit(off_2, 80)
find_distances(off_2, 46, 2, unit_2)


(278877209.3023256, 10568707.957255056)

In [49]:
mode_dist(L=(60,.5))


(249827048.33333334, 2081892.0694444445)

In [127]:
doppler_data = off_1[50:300]
doppler_peaks, _ = find_peaks(doppler_data, distance=10, height=50)


In [144]:
m = doppler_peaks[0]
o = np.median(doppler_data)
def gauss(x, s, A):
    return A*np.exp(-(x-m)**2/(2*s**2)) + o

doppler, ddoppler = curve_fit(gauss, doppler_peaks, doppler_data[doppler_peaks],
                              p0=(20, doppler_data[doppler_peaks[0]]))
ddoppler = np.sqrt(np.diag(ddoppler))

In [173]:
plot_peaks(doppler_data, doppler_peaks)
xs = np.linspace(0, 250, 1000)
plt.plot(xs, gauss(xs, *doppler), label='Einhuellende')
plt.legend()
plt.show()

sigma = SecondaryValue('s*u')(s=(doppler[0], 40), u=unit_1)

In [174]:
T = SecondaryValue('(s*c/n0)^2*m/k')(c=299792458, s=sigma, m=3.35092e-26, n0=4.73755e14, k=1.380649e-23)
T

(113.22422437226905, 170.7171426794478)

In [172]:
sigma[0]*2*np.sqrt(2*np.log(2))

1346844852.1465542

In [158]:
doppler, ddoppler, sigma

(array([53.11198736, 47.34934225]),
 array([8.08987576, 4.31644846]),
 (341320515.0376786, 129049367.49361125))